# Analysis of Duchess' Eating Habits
William Reames

## Imports

In [1]:
import pandas as pd
import numpy as np

## Variables Used in During Analysis

In [2]:
minimum_satisfactory_immediate_eaten = 0.40
minimum_satisfactory_total_eaten = 0.80
pd.options.display.max_rows = None
pd.options.display.max_columns = None

## Importing the Dataset

In [3]:
df = pd.read_csv('../data/duchess-eating-habits.csv')

In [4]:
df.head()

,Date,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,Immediate % Eaten,Total % Eaten,Notes
0,10/25/2023,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,5%,10%,NaN
1,10/25/2023,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,60%,100%,NaN
2,10/26/2023,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,25%,80%,NaN
3,10/26/2023,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,30%,95%,NaN
4,10/27/2023,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,15%,60%,NaN


## Cleaning the Data

Dropping unecessary data

In [5]:
df = df[~df['Meal'].isnull()]

In [6]:
df = df.drop('Notes', axis=1)

In [7]:
df = df.drop('Date', axis=1)

In [8]:
df.head()

,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,Immediate % Eaten,Total % Eaten
0,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,5%,10%
1,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,60%,100%
2,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,25%,80%
3,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,30%,95%
4,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,15%,60%


Converting % eaten data to integer values

In [9]:
df['immediate_eaten'] = pd.to_numeric(df['Immediate % Eaten'].str.replace('%', '')) / 100
df = df.drop('Immediate % Eaten', axis=1)

In [10]:
df['total_eaten'] = pd.to_numeric(df['Total % Eaten'].str.replace('%', '')) / 100
df = df.drop('Total % Eaten', axis=1)

In [11]:
df.head()

,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,immediate_eaten,total_eaten
0,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,0.05,0.10
1,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,0.60,1.00
2,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,0.25,0.80
3,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,0.30,0.95
4,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,0.15,0.60


Determining food satisfaction levels

In [12]:
df['satisfactory_immediate_eaten'] = df['immediate_eaten'] >= minimum_satisfactory_immediate_eaten
df['satisfactory_total_eaten'] = df['total_eaten'] >= minimum_satisfactory_total_eaten

In [13]:
df.head()

,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,immediate_eaten,total_eaten,satisfactory_immediate_eaten,satisfactory_total_eaten
0,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,0.05,0.10,False,False
1,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,0.60,1.00,True,True
2,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,0.25,0.80,False,True
3,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,0.30,0.95,False,True
4,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,0.15,0.60,False,False


Adding relevant data to each row based on previous rows

In [14]:
# Add a column for the previous meal's satisfactory_total_eaten value
# Add a column for the number of times this flavor was served during the previous 6 meals

Creating binary encodings

In [15]:
df['lunch'] = pd.get_dummies(df['Meal'])['Lunch']
df = df.drop('Meal', axis=1)

In [16]:
df['initially_cold'] = pd.get_dummies(df['Initial Temperature'])['Cold']
df = df.drop('Initial Temperature', axis=1)

In [17]:
df.head()

,Brand,Flavor,Type,Microwaved,Added Water,immediate_eaten,total_eaten,satisfactory_immediate_eaten,satisfactory_total_eaten,lunch,initially_cold
0,Friskies,Salmon & Gravy,Chunky extra gravy,False,False,0.05,0.10,False,False,1,1
1,Friskies,Turkey & Cheese,Shreds,False,False,0.60,1.00,True,True,0,0
2,Friskies,Turkey & Cheese,Shreds,False,False,0.25,0.80,False,True,1,1
3,Friskies,Whitefish & Sardines,Shreds,False,False,0.30,0.95,False,True,0,0
4,Friskies,Whitefish & Sardines,Shreds,False,False,0.15,0.60,False,False,1,1


Creating one-hot encodings

In [18]:
df['Flavor'] = df['Flavor'].str.lower()
df['Flavor'] = df['Flavor'].str.replace('ocean whitefish', 'whitefish')
df['Flavor'] = df['Flavor'].str.replace('grain free salmon', 'salmon')

In [19]:
df[['flavor_1', 'flavor_2']] = df['Flavor'].str.split(' & ', expand=True)
df = df.drop('Flavor', axis=1)

In [20]:
df = pd.concat([df, pd.get_dummies(df['flavor_1'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['flavor_2'])], axis=1)
df = df.drop('flavor_1', axis=1)
df = df.drop('flavor_2', axis=1)

In [23]:
df.head()

,Brand,Type,Microwaved,Added Water,immediate_eaten,total_eaten,satisfactory_immediate_eaten,satisfactory_total_eaten,lunch,initially_cold,chicken,country style,mariner's catch,mixed grill,salmon,turkey,whitefish,cheese,giblets,gravy,sardines,tuna
0,Friskies,Chunky extra gravy,False,False,0.05,0.10,False,False,1,1,0,0,0,0,1,0,0,0,0,1,0,0
1,Friskies,Shreds,False,False,0.60,1.00,True,True,0,0,0,0,0,0,0,1,0,1,0,0,0,0
2,Friskies,Shreds,False,False,0.25,0.80,False,True,1,1,0,0,0,0,0,1,0,1,0,0,0,0
3,Friskies,Shreds,False,False,0.30,0.95,False,True,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,Friskies,Shreds,False,False,0.15,0.60,False,False,1,1,0,0,0,0,0,0,1,0,0,0,1,0
